In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_cbMSIn0d0014nonInDI_norm.csv")
diDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_nonInDI_cbMSIn0d0014nonInDI_norm.csv")

In [4]:
trainDEFSDf

,pixel_id,311.168,269.249,215.033,295.227,883.533,309.172,738.505,435.296,280.236,241.217,311.223,339.2,353.201,325.184,250.145,514.284,265.148,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.318766,0.166427,-0.119110,-0.066937,-0.070112,0.384434,-0.143208,0.118236,-0.082543,0.226676,-0.001238,0.230791,0.249851,0.293677,0.126166,-0.040413,0.322943,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.440857,0.435660,-0.038277,-0.066937,-0.037918,0.479902,-0.160188,0.058335,-0.102396,0.292328,-0.001238,0.312519,0.313742,0.399221,0.300477,-0.040413,0.392732,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.297861,0.183927,-0.083686,-0.066937,-0.067620,0.317128,-0.150589,-0.029346,-0.088092,0.210673,-0.001238,0.229711,0.257741,0.267807,0.208800,-0.040413,0.315278,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.276325,0.127673,-0.042443,-0.066937,-0.052332,0.299088,-0.142959,-0.017979,-0.106704,0.272887,-0.001238,0.221054,0.208527,0.247314,0.187171,-0.040413,0.303845,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.172636,0.221349,-0.131908,-0.066937,-0.050631,0.219242,-0.145985,0.140151,-0.064397,0.166874,-0.001238,0.138944,0.194243,0.151402,0.094791,-0.040413,0.213334,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.035497,-0.299458,0.004159,-0.066937,-0.140680,0.062989,-0.200210,0.242831,-0.173629,-0.248629,-0.001238,0.042851,0.066512,0.035224,0.038177,0.212906,0.036063,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.021830,-0.299458,-0.091240,-0.066937,-0.060323,0.074326,-0.191599,0.300569,-0.195069,0.152803,-0.001238,0.050360,0.070399,0.037017,0.210630,-0.040413,0.018601,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.030910,0.367236,-0.093395,-0.066937,-0.089147,0.044593,-0.114107,0.195878,-0.156602,-0.248629,-0.001238,0.046832,0.103164,0.037407,0.143304,-0.040413,-0.052544,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.045528,-0.299458,-0.122677,-0.066937,-0.161395,0.060010,-0.138312,0.149756,-0.162997,-0.248629,-0.001238,0.033926,-0.067441,0.036386,0.189039,-0.040413,0.048053,1


In [5]:
can = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(trainDEFSDf.columns))-1]:
    can.append(list(trainDEFSDf.columns)[i])

dican = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(diDEFSDf.columns))-1]:
    dican.append(list(diDEFSDf.columns)[i])

In [6]:
can

['pixel_id',
 '311.168',
 '269.249',
 '215.033',
 '295.227',
 '883.533',
 '309.172',
 '738.505',
 '435.296',
 '250.145',
 '514.284',
 'type']

In [7]:
dican

['pixel_id',
 '311.169',
 '269.249',
 '215.032',
 '295.228',
 '883.538',
 '309.17',
 '738.508',
 '435.297',
 '250.145',
 '514.284',
 'type']

In [8]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_cbMSIn0d0014nonInDI_norm.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
trainDEFSDf = trainDEFSDf[can]
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [9]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ext_cbMSIn0d0014nonInDI_norm.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
extDEFSDf = extDEFSDf[can]
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [10]:
## input ingested set ## 97035 x 20 df
ingestedDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ingested_cbMSIn0d0014nonInDI_norm.csv")
#ingestedDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
ingestedDEFSDf = ingestedDEFSDf[can]
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [11]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_FNA_cbMSIn0d0014nonInDI_norm.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
fnaDEFSDf = fnaDEFSDf[can]
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [12]:
## input DirectIn set ## 88701 x 20 df
diDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_nonInDI_cbMSIn0d0014nonInDI_norm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 3027, 1: 3030
diDEFSDf = diDEFSDf[dican]
diDEFSDf = diDEFSDf.rename(columns={"311.169": "311.168", "215.032":"215.033", "295.228":"295.227", "883.538":"883.533", 
                                    "309.17":"309.172", "738.508":"738.505", "435.297":"435.296", "311.222":"311.223"})
Yy_DI = diDEFSDf["type"]  # 1.0005; 0.9995
sampleDiW = []
for w in Yy_DI:
    if w == 0:
        sampleDiW.append(1.0005)
    elif w == 1:
        sampleDiW.append(0.9995)


In [13]:
diDEFSDf

,pixel_id,311.168,269.249,215.033,295.227,883.533,309.172,738.505,435.296,250.145,514.284,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.005233,0.142288,0.041741,0.003879,0.080422,-0.001508,0.061910,0.126365,0.467350,0.036504,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.089591,0.156412,0.177400,0.001265,0.169447,-0.001508,0.128052,0.146025,0.583306,0.099624,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.032985,0.134206,0.036982,-0.069655,0.124404,-0.001508,0.029039,0.077323,0.465193,0.056832,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.081001,0.235152,0.064486,0.057514,0.131518,-0.001508,0.108169,0.193505,0.550698,0.090345,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.132548,0.236938,0.155168,0.080273,0.241694,-0.001508,0.236299,0.205055,0.645501,0.147339,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.002046,-0.209455,0.129994,-0.039157,-0.040097,-0.001508,0.023320,0.147663,-0.089365,0.163274,1
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.313188,-0.217983,-0.306237,-0.299289,-0.270588,-0.001508,-0.280477,-0.277348,-0.025231,-0.251938,1
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.158181,-0.037213,0.001820,-0.048526,-0.042687,-0.001508,-0.084838,-0.038916,-0.089365,0.091535,1
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.130111,-0.100314,0.068322,-0.107947,-0.004388,-0.001508,-0.068925,-0.070150,-0.010167,-0.018033,1


In [14]:
## define functions for performace evaluation ##

# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [15]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,311.168,269.249,215.033,295.227,883.533,309.172,738.505,435.296,250.145,514.284
0,0.318766,0.166427,-0.119110,-0.066937,-0.070112,0.384434,-0.143208,0.118236,0.126166,-0.040413
1,0.440857,0.435660,-0.038277,-0.066937,-0.037918,0.479902,-0.160188,0.058335,0.300477,-0.040413
2,0.297861,0.183927,-0.083686,-0.066937,-0.067620,0.317128,-0.150589,-0.029346,0.208800,-0.040413
3,0.276325,0.127673,-0.042443,-0.066937,-0.052332,0.299088,-0.142959,-0.017979,0.187171,-0.040413
4,0.172636,0.221349,-0.131908,-0.066937,-0.050631,0.219242,-0.145985,0.140151,0.094791,-0.040413
...,...,...,...,...,...,...,...,...,...,...
90955,0.035497,-0.299458,0.004159,-0.066937,-0.140680,0.062989,-0.200210,0.242831,0.038177,0.212906
90956,0.021830,-0.299458,-0.091240,-0.066937,-0.060323,0.074326,-0.191599,0.300569,0.210630,-0.040413
90957,0.030910,0.367236,-0.093395,-0.066937,-0.089147,0.044593,-0.114107,0.195878,0.143304,-0.040413
90958,0.045528,-0.299458,-0.122677,-0.066937,-0.161395,0.060010,-0.138312,0.149756,0.189039,-0.040413


In [16]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [17]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [18]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf]).set_index('pixel_id')

In [19]:
df_ROI_for_ML_Opti_ingested = ingestedDEFSDf.set_index('pixel_id')

In [20]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [21]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [22]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

In [23]:
df_ROI_for_ML_Opti_di = diDEFSDf.set_index('pixel_id')

In [24]:
df_ROI_for_ML_Opti_di.describe()

,311.168,269.249,215.033,295.227,883.533,309.172,738.505,435.296,250.145,514.284,type
count,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6057.000000
mean,2.786097e-18,6.620646e-17,-1.434290e-17,-1.438873e-18,4.710337e-16,-5.057534e-18,1.524563e-16,7.979419e-16,-4.598716e-15,2.271036e-17,0.500248
std,1.259709e-01,1.439456e-01,1.644742e-01,1.354221e-01,1.830145e-01,3.246305e-02,1.277902e-01,2.078016e-01,1.914162e-01,1.330269e-01,0.500041
min,-3.936976e-01,-4.211962e-01,-3.062371e-01,-4.545066e-01,-2.705880e-01,-1.508251e-03,-2.804766e-01,-2.773479e-01,-8.936520e-02,-3.327737e-01,0.000000
25%,-7.672668e-02,-6.810890e-02,-9.447482e-02,-6.188994e-02,-1.155605e-01,-1.508251e-03,-5.989332e-02,-2.773479e-01,-8.936520e-02,-6.633072e-02,0.000000
50%,9.552356e-03,-6.986512e-03,1.677855e-02,1.774746e-02,3.213359e-02,-1.508251e-03,1.466426e-02,6.586256e-03,-8.936520e-02,-1.852649e-02,1.000000
75%,9.000432e-02,5.958404e-02,1.127961e-01,7.451773e-02,1.324455e-01,-1.508251e-03,6.962831e-02,1.670720e-01,-1.214997e-03,8.939424e-02,1.000000
max,6.063024e-01,5.788038e-01,6.937629e-01,5.454934e-01,7.294120e-01,9.984917e-01,7.195234e-01,7.226521e-01,9.106348e-01,6.672263e-01,1.000000


## 2.2. PCA Analysis

In [39]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_di):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_di.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/cbMSIn_0d001/PCA_di.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [ ]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [41]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [42]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [43]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [44]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [45]:
y_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[-1]]

In [46]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [47]:
y_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[-1]]

In [48]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [49]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [50]:
X_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[:-1]]

In [51]:
y_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[-1]]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [52]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.1. Logistic Regression

In [53]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [54]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.8390982668238988
Training Set MCC: 0.6791888928889686
Training Set Accuracy: 0.8397207563764292
              precision    recall  f1-score   support

           0       0.85      0.84      0.85     47449
           1       0.83      0.84      0.83     43511

    accuracy                           0.84     90960
   macro avg       0.84      0.84      0.84     90960
weighted avg       0.84      0.84      0.84     90960

[[39983  7466]
 [ 7113 36398]]
Ext Val Set F1: 0.8155151465528109
Ext Val Set MCC: 0.6047170472841887
Ext Val Set Accuracy: 0.7976954732510289
              precision    recall  f1-score   support

           0       0.88      0.68      0.76      2943
           1       0.75      0.91      0.82      3132

    accuracy                           0.80      6075
   macro avg       0.81      0.79      0.79      6075
weighted avg       0.81      0.80      0.79      6075

[[1995  948]
 [ 281 2851]]
5F-CV: 0.628095702338524
FNA Set F1: 0.74656967408123
FNA Set

### 3.2.2. SVM

In [55]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [56]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.8797283533548552
Training Set MCC: 0.7565608917496852
Training Set Accuracy: 0.8775835532102023
              precision    recall  f1-score   support

           0       0.90      0.87      0.88     47449
           1       0.86      0.89      0.87     43511

    accuracy                           0.88     90960
   macro avg       0.88      0.88      0.88     90960
weighted avg       0.88      0.88      0.88     90960

[[41046  6403]
 [ 4732 38779]]
Ext Val Set F1: 0.8277768644493677
Ext Val Set MCC: 0.6533037808217003
Ext Val Set Accuracy: 0.8268312757201646
              precision    recall  f1-score   support

           0       0.82      0.82      0.82      2943
           1       0.83      0.83      0.83      3132

    accuracy                           0.83      6075
   macro avg       0.83      0.83      0.83      6075
weighted avg       0.83      0.83      0.83      6075

[[2413  530]
 [ 522 2610]]
5F-CV: 0.9077546357959191
FNA Set F1: 0.6315512601979377
FNA 

### 3.2.3. GBM

In [57]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [58]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.3387704774653022
Training Set MCC: 0.10187904349252903
Training Set Accuracy: 0.5535729991204925
              precision    recall  f1-score   support

           0       0.55      0.85      0.66     47449
           1       0.58      0.24      0.34     43511

    accuracy                           0.55     90960
   macro avg       0.56      0.54      0.50     90960
weighted avg       0.56      0.55      0.51     90960

[[40107  7342]
 [33265 10246]]
Ext Val Set F1: 0.19208499371408716
Ext Val Set MCC: -0.11284184526582608
Ext Val Set Accuracy: 0.44724279835390945
              precision    recall  f1-score   support

           0       0.46      0.79      0.58      2943
           1       0.39      0.13      0.19      3132

    accuracy                           0.45      6075
   macro avg       0.42      0.46      0.39      6075
weighted avg       0.42      0.45      0.38      6075

[[2313  630]
 [2728  404]]
5F-CV: 0.6475279700575391
FNA Set F1: 0.4577778617935277

### 3.2.4. Stacking

In [59]:
class StackingRegressor():
    
    def __init__(self, learners):
        self.level_sizes = []
        self.learners = []
        
        for learning_level in learners:
            self.level_sizes.append(len(learning_level))
            level_learners = []
            
            for learner in learning_level:
                level_learners.append(deepcopy(learner))
            self.learners.append(level_learners)
            
            
    def fit(self, x, y):
        
        meta_data = [x,x,x]
        meta_targets = [y,y,y]
        
        for i in range(len(self.learners)):
            level_size = self.level_sizes[i]
            
            data_z = np.zeros((level_size, len(x)))
            target_z = np.zeros(len(x))
            
            train_x = meta_data[i]
            train_y = meta_targets[i]
            
            # Define number of folds
            num_folds = 5
            
            # Create the k-fold cross-validation object
            KF = KFold(n_splits=num_folds)
            m = 0

            # Loop over each fold of the cross-validation
            for train_indices, test_indices in KF.split(x):
                
                for j in range(len(self.learners[i])):
                    train_x = pd.DataFrame(train_x)
                    train_y = pd.DataFrame(train_y)

                    learner = self.learners[i][j]
                    learner.fit(train_x.iloc[train_indices], train_y.iloc[train_indices])
                    p = learner.predict(train_x.iloc[test_indices])
                    data_z[j][m: m+len(test_indices) ] = p


                train_y = pd.DataFrame(train_y)
                train_y_array = (np.array(train_y)).reshape(-1,)
                zty_ind = (np.array(train_y.iloc[test_indices])).reshape(-1,)
                target_z[m: m+len(test_indices)] = train_y_array[zty_ind]
                m += len(test_indices)
                

            data_z = data_z.transpose()
            meta_data.append(data_z)
            meta_targets.append(target_z)
            
            
            for learner in self.learners[i]:
                train_x = pd.DataFrame(train_x)
                train_y = pd.DataFrame(train_y)
                learner.fit(train_x, train_y)
                
                
    def predict(self, x):
        
        meta_data = [x,x,x]
        for i in range(len(self.learners)):
            
            level_size = self.level_sizes[i]
            
            data_z = np.zeros((level_size, len(x)))
            
            test_x = meta_data[i]
            
            for j in range(len(self.learners[i])):
                
                learner = self.learners[i][j]
                predictions = learner.predict(test_x)
                data_z[j] = predictions
                
            
            data_z = data_z.transpose()
            meta_data.append(data_z)
            
        return meta_data


In [60]:
## Build stacking Model

def Stacking_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    #meta_learner = LinearDiscriminantAnalysis()  # 0.90,0.80; 0.75,0.51; 0.65,0.27; 0.64,0.26
    #meta_learner = QuadraticDiscriminantAnalysis()  # 0.93,0.86; 0.67,0.39; 0.67,0.15; 0.68,0.20
    
    #meta_learner = LogisticRegression()  # 0.90,0.79; 0.82,0.65; 0.64,0.24; 0.64;0.27
    #meta_learner = LogisticRegression(C=1, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=1: 0.90,0.79; 0.82,0.65; 0.65,0.25; 0.65;0.27
    #meta_learner = LogisticRegression(C=0.1, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.1: 0.90,0.79; 0.82,0.65; 0.65,0.25; 0.65;0.28
    #meta_learner = LogisticRegression(C=0.01, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.01: 0.89,0.78; 0.81,0.63; 0.63,0.22; 0.65;0.29
    #meta_learner = LogisticRegression(C=0.0075, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0075: 0.89,0.77; 0.81,0.63; 0.62,0.20; 0.65;0.29
    #meta_learner = LogisticRegression(C=0.005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.005: 0.88,0.76; 0.82,0.64; 0.60,0.17; 0.66;0.31
    #meta_learner = LogisticRegression(C=0.0025, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0025: 0.86,0.72; 0.83,0.66; 0.53,0.02; 0.64;0.31
    #meta_learner = LogisticRegression(C=0.0015, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0015: 0.85,0.70; 0.80,0.60; 0.55,0.09; 0.64;0.30
    #meta_learner = LogisticRegression(C=0.0013, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0013: 0.85,0.70; 0.78,0.56; 0.58,0.16; 0.63;0.29
    #meta_learner = LogisticRegression(C=0.0012, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0012: 0.85,0.69; 0.77,0.52; 0.60,0.22; 0.63;0.27
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0011: 0.85,0.69; 0.77,0.51; 0.64,0.30; 0.62;0.25
    #meta_learner = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.001: 0.84,0.68; 0.76,0.49; 0.68,0.38; 0.61;0.22
    #meta_learner = LogisticRegression(C=0.0005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0005: 0.83,0.66; 0.74,0.45; 0.65,0.37; 0.60;0.18
    
    #meta_learner = LinearSVC()  # 0.90,0.80; 0.82,0.65; 0.64,0.24; 0.65,0.27
    #meta_learner = KNeighborsClassifier()  # 0.98,0.96; 0.65,0.30; 0.68,0.21; 0.52,-0.05
    #meta_learner = DecisionTreeClassifier()  # 0.94,0.87; 0.54,0.15; 0.66,0.11; 0.46,-0.11
    #meta_learner = RandomForestClassifier()  # 0.98,0.95; 0.54,0.20; 0.66,0.11; 0.49;-0.01
    #meta_learner = GradientBoostingClassifier()  # 0.97,0.94; 0.56,0.27; 0.66,0.13; 0.48,-0.09
    #meta_learner = AdaBoostClassifier()  # 0.96,0.93; 0.69,0.42; 0.62,0.10; 0.54,0.11
    #meta_learner = xgb.XGBClassifier()  # 0.98,0.95; 0.59,0.30; 0.66,0.10; 0.44,-0.12

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(X_train)[-1]
    print("Training Set F1:", f1_score(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XVal)[-1]
    print("Ext Val Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [61]:
Stacking_func()

Training Set F1: 0.8406163945092073
Training Set MCC: 0.682009460263964
Training Set Accuracy: 0.8411059806508355
              precision    recall  f1-score   support

           0       0.85      0.84      0.85     47449
           1       0.83      0.84      0.83     43511

    accuracy                           0.84     90960
   macro avg       0.84      0.84      0.84     90960
weighted avg       0.84      0.84      0.84     90960

[[40020  7429]
 [ 7024 36487]]
Ext Val Set F1: 0.832423918527915
Ext Val Set MCC: 0.6444376315589516
Ext Val Set Accuracy: 0.8189300411522634
              precision    recall  f1-score   support

           0       0.89      0.72      0.79      2943
           1       0.77      0.91      0.84      3132

    accuracy                           0.82      6075
   macro avg       0.83      0.82      0.82      6075
weighted avg       0.83      0.82      0.82      6075

[[2110  833]
 [ 267 2865]]
FNA Set F1: 0.7107423761966096
FNA Set MCC: 0.4393255235974306


### 3.2.5. Voting

In [62]:
# get a voting ensemble of models
def get_voting():
    models = list()
    # Define the base models
    models.append(("m1", LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m2", LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m3", GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)))

    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='hard')
    return ensemble

# get a list of models to evaluate
def get_models():
    models = dict()
    models['m1'] = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m2'] = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m3'] = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    models['hard_voting'] = get_voting()
    return models


# evaluate a give model using cross-validation
def evaluate_model(model, XX, yy):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = cross_val_score(model, XX, yy, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
    return scores


In [63]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [65]:
# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, XX=X_ingested, yy=y_ingested)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    
# plot model performance for comparison
plt.subplots(dpi = 600)
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('F1 score')
plt.xlabel('Predictive Models')
plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/cbMSIn_0d001/voting.jpg", bbox_inches = 'tight')

>m1 0.812 (0.003)
>m2 0.935 (0.007)
>m3 0.676 (0.127)
>hard_voting 0.904 (0.019)


In [66]:
def Voting_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [67]:
Voting_func()

Training Set F1: 0.8621242268490743
Training Set MCC: 0.7249109771465304
Training Set Accuracy: 0.8625549692172384
              precision    recall  f1-score   support

           0       0.87      0.86      0.87     47449
           1       0.85      0.86      0.86     43511

    accuracy                           0.86     90960
   macro avg       0.86      0.86      0.86     90960
weighted avg       0.86      0.86      0.86     90960

[[41036  6413]
 [ 6089 37422]]
Ext Val Set F1: 0.8259854700197835
Ext Val Set MCC: 0.6607549001846557
Ext Val Set Accuracy: 0.8293004115226338
              precision    recall  f1-score   support

           0       0.81      0.85      0.83      2943
           1       0.85      0.81      0.83      3132

    accuracy                           0.83      6075
   macro avg       0.83      0.83      0.83      6075
weighted avg       0.83      0.83      0.83      6075

[[2511  432]
 [ 605 2527]]
5F-CV: 0.8551381473771457
FNA Set F1: 0.7068452106249302
FNA 

In [68]:
import joblib as jl

In [69]:
models = get_models()
    
ensemble = models["hard_voting"]
ensemble.fit(X_ingested, y_ingested)

VotingClassifier(estimators=[('m1',
                              LogisticRegression(C=0.001,
                                                 class_weight={0: 0.9628,
                                                               1: 1.0402},
                                                 max_iter=5000, penalty='l1',
                                                 random_state=42,
                                                 solver='liblinear')),
                             ('m2',
                              LinearSVC(C=461,
                                        class_weight={0: 0.9628, 1: 1.0402},
                                        random_state=42)),
                             ('m3',
                              GradientBoostingClassifier(learning_rate=4,
                                                         max_depth=7,
                                                         min_samples_leaf=4,
                                                         min_sampl

In [70]:
modelSavePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\model3to1Voting_cbMSIn0d001.joblib"
jl.dump(ensemble, modelSavePath, compress = 5)

['H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\cbMSIn_0d001\\model3to1Voting_cbMSIn0d001.joblib']

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### 3.3.1. Logistic Regression

In [71]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [72]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.859050437745964
Training Set MCC: 0.7146122915983197
Training Set Accuracy: 0.8566072999120492
              precision    recall  f1-score   support

           0       0.88      0.84      0.86     47449
           1       0.84      0.87      0.85     43511

    accuracy                           0.86     90960
   macro avg       0.86      0.86      0.86     90960
weighted avg       0.86      0.86      0.86     90960

[[40044  7405]
 [ 5638 37873]]
Ext Val Set F1: 0.7461016750410112
Ext Val Set MCC: 0.4571267286244885
Ext Val Set Accuracy: 0.7280658436213991
              precision    recall  f1-score   support

           0       0.76      0.64      0.70      2943
           1       0.71      0.81      0.75      3132

    accuracy                           0.73      6075
   macro avg       0.73      0.73      0.73      6075
weighted avg       0.73      0.73      0.73      6075

[[1897 1046]
 [ 606 2526]]
5F-CV: 0.7508338046408912
FNA Set F1: 0.5138786540927918
FNA S

### 3.3.2. SVM

In [73]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [74]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.936826200403876
Training Set MCC: 0.8786876453940262
Training Set Accuracy: 0.9398636763412489
              precision    recall  f1-score   support

           0       0.92      0.97      0.94     47449
           1       0.96      0.91      0.94     43511

    accuracy                           0.94     90960
   macro avg       0.94      0.94      0.94     90960
weighted avg       0.94      0.94      0.94     90960

[[45891  1558]
 [ 3912 39599]]
Ext Val Set F1: 0.5463637210755545
Ext Val Set MCC: 0.1136442372665275
Ext Val Set Accuracy: 0.5560493827160494
              precision    recall  f1-score   support

           0       0.54      0.58      0.56      2943
           1       0.57      0.53      0.55      3132

    accuracy                           0.56      6075
   macro avg       0.56      0.56      0.56      6075
weighted avg       0.56      0.56      0.56      6075

[[1706 1237]
 [1460 1672]]
5F-CV: 0.8985142905056881
FNA Set F1: 0.605895184096619
FNA Se

### 3.3.3. GBM

In [75]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [76]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.7741684531291156
Training Set MCC: 0.5971718001016285
Training Set Accuracy: 0.7977572559366755
              precision    recall  f1-score   support

           0       0.77      0.88      0.82     47449
           1       0.85      0.71      0.77     43511

    accuracy                           0.80     90960
   macro avg       0.81      0.79      0.79     90960
weighted avg       0.80      0.80      0.80     90960

[[41832  5617]
 [12779 30732]]
Ext Val Set F1: 0.5172605730534929
Ext Val Set MCC: 0.06087729188052219
Ext Val Set Accuracy: 0.5295473251028806
              precision    recall  f1-score   support

           0       0.51      0.56      0.53      2943
           1       0.55      0.50      0.52      3132

    accuracy                           0.53      6075
   macro avg       0.53      0.53      0.53      6075
weighted avg       0.53      0.53      0.53      6075

[[1641 1302]
 [1556 1576]]
5F-CV: 0.7273904539489974
FNA Set F1: 0.5601559761648502
FNA

### 3.3.4. Stacking

In [77]:
## Build stacking Model

def Stacking_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(X_train)[-1]
    print("Training Set F1:", f1_score(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XVal)[-1]
    print("Ext Val Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [78]:
Stacking_func()

Training Set F1: 0.8662651830389484
Training Set MCC: 0.7291963039236078
Training Set Accuracy: 0.8638852242744063
              precision    recall  f1-score   support

           0       0.88      0.85      0.87     47449
           1       0.84      0.88      0.86     43511

    accuracy                           0.86     90960
   macro avg       0.86      0.86      0.86     90960
weighted avg       0.86      0.86      0.86     90960

[[40381  7068]
 [ 5313 38198]]
Ext Val Set F1: 0.7507740699813095
Ext Val Set MCC: 0.48543084825473565
Ext Val Set Accuracy: 0.7432098765432099
              precision    recall  f1-score   support

           0       0.75      0.71      0.73      2943
           1       0.74      0.78      0.76      3132

    accuracy                           0.74      6075
   macro avg       0.74      0.74      0.74      6075
weighted avg       0.74      0.74      0.74      6075

[[2082  861]
 [ 699 2433]]
FNA Set F1: 0.4948210434864645
FNA Set MCC: -0.0461151049017

### 3.3.5. Voting

In [79]:
def Voting_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [80]:
Voting_func()

Training Set F1: 0.9290585732655074
Training Set MCC: 0.8639791582334887
Training Set Accuracy: 0.9325087950747581
              precision    recall  f1-score   support

           0       0.91      0.96      0.94     47449
           1       0.96      0.90      0.93     43511

    accuracy                           0.93     90960
   macro avg       0.93      0.93      0.93     90960
weighted avg       0.93      0.93      0.93     90960

[[45618  1831]
 [ 4308 39203]]
Ext Val Set F1: 0.675045534099414
Ext Val Set MCC: 0.3331311496465998
Ext Val Set Accuracy: 0.6671604938271605
              precision    recall  f1-score   support

           0       0.66      0.64      0.65      2943
           1       0.67      0.69      0.68      3132

    accuracy                           0.67      6075
   macro avg       0.67      0.67      0.67      6075
weighted avg       0.67      0.67      0.67      6075

[[1882 1061]
 [ 961 2171]]
5F-CV: 0.8796024401234556
FNA Set F1: 0.5878371640587023
FNA S

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

### 3.4.1. Logistic Regression

In [81]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [82]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
5F-CV: 0.0
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              

### 3.4.2. SVM

In [83]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [84]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.5361303550905401
Training Set MCC: -0.009055524021052778
Training Set Accuracy: 0.49175461741424803
              precision    recall  f1-score   support

           0       0.52      0.41      0.46     47449
           1       0.47      0.58      0.52     43511

    accuracy                           0.49     90960
   macro avg       0.50      0.50      0.49     90960
weighted avg       0.50      0.49      0.49     90960

[[19362 28087]
 [18143 25368]]
Ext Val Set F1: 0.9527368264110506
Ext Val Set MCC: 0.9103290940063884
Ext Val Set Accuracy: 0.9532510288065844
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      2943
           1       0.99      0.92      0.95      3132

    accuracy                           0.95      6075
   macro avg       0.95      0.95      0.95      6075
weighted avg       0.96      0.95      0.95      6075

[[2902   41]
 [ 243 2889]]
5F-CV: 0.8710870905637776
FNA Set F1: 0.44731069235589443

### 3.4.3. GBM

In [85]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_gbm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_gbm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [86]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.5863644483618763
Training Set MCC: 0.08615634286831296
Training Set Accuracy: 0.5374450307827616
              precision    recall  f1-score   support

           0       0.57      0.44      0.50     47449
           1       0.51      0.65      0.57     43511

    accuracy                           0.54     90960
   macro avg       0.54      0.54      0.53     90960
weighted avg       0.55      0.54      0.53     90960

[[20642 26807]
 [15267 28244]]
Ext Val Set F1: 0.8858235368847232
Ext Val Set MCC: 0.7679336774159764
Ext Val Set Accuracy: 0.8842798353909465
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2943
           1       0.88      0.90      0.89      3132

    accuracy                           0.88      6075
   macro avg       0.88      0.88      0.88      6075
weighted avg       0.88      0.88      0.88      6075

[[2546  397]
 [ 306 2826]]
5F-CV: 0.7432621722756714
FNA Set F1: 0.5866301920799357
FNA

### 3.4.4. Stacking

In [87]:
## Build stacking Model

def Stacking_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(XVal)[-1]
    print("Training Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XTrain)[-1]
    print("Ext Val Set F1:", f1_score(yTrain, model_sc.predict(XTrain)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_sc.predict(XTrain)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [88]:
Stacking_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              precision  

### 3.4.5. Voting

In [89]:
def Voting_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(XVal)
    print("Training Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, ensemble.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, ensemble.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [90]:
Voting_func()

Training Set F1: 0.40068780281187855
Training Set MCC: -0.08164266841007049
Training Set Accuracy: 0.46427000879507474
              precision    recall  f1-score   support

           0       0.49      0.56      0.52     47449
           1       0.43      0.36      0.39     43511

    accuracy                           0.46     90960
   macro avg       0.46      0.46      0.46     90960
weighted avg       0.46      0.46      0.46     90960

[[26521 20928]
 [27802 15709]]
Ext Val Set F1: 0.9118366796605383
Ext Val Set MCC: 0.8459695879583415
Ext Val Set Accuracy: 0.9160493827160494
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      2943
           1       0.99      0.84      0.91      3132

    accuracy                           0.92      6075
   macro avg       0.92      0.92      0.92      6075
weighted avg       0.93      0.92      0.92      6075

[[2920   23]
 [ 487 2645]]
5F-CV: 0.7638403247393223
FNA Set F1: 0.43991384396353395

# 4. Model Evaluation

## 4.1 Prediction

In [91]:
## deploy models for training set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\model3to1Voting_cbMSIn0d001.joblib")
#
## deploy model ##
predicted0n1_train = model_voting.predict(X_train)
#predicted0n1_trainProba = model_voting.predict_proba(X_train)         
trainDEFSDf["predicted0n1"] = predicted0n1_train
T_ = []
F_ = []
for i in range(len(trainDEFSDf)):
    if trainDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif trainDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
trainDEFSDf["T"] = T_
trainDEFSDf["F"] = F_
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\df_train_cbMSIn0d001_norm_0n1.csv"
trainDEFSDf.to_csv(savePath)

In [92]:
## deploy models for Ext Val set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\model3to1Voting_cbMSIn0d001.joblib")
#
## deploy model ##
predicted0n1_ext = model_voting.predict(X_val)
extDEFSDf["predicted0n1"] = predicted0n1_ext
T_ = []
F_ = []
for i in range(len(extDEFSDf)):
    if extDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif extDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
extDEFSDf["T"] = T_
extDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\df_ext_cbMSIn0d001_norm_0n1.csv"
extDEFSDf.to_csv(savePath)

In [93]:
## deploy models for FNA set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\model3to1Voting_cbMSIn0d001.joblib")
#
## deploy model ##
predicted0n1_FNA = model_voting.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
T_ = []
F_ = []
for i in range(len(fnaDEFSDf)):
    if fnaDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif fnaDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
fnaDEFSDf["T"] = T_
fnaDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\df_FNA_cbMSIn0d001_norm_0n1.csv"
fnaDEFSDf.to_csv(savePath)

In [94]:
## deploy models for DirectInfusion set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\model3to1Voting_cbMSIn0d001.joblib")
#
## deploy model ##
predicted0n1_Di = model_voting.predict(X_di)
diDEFSDf["predicted0n1"] = predicted0n1_Di
T_ = []
F_ = []
for i in range(len(diDEFSDf)):
    if diDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif diDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
diDEFSDf["T"] = T_
diDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\cbMSIn_0d001\df_Di_cbMSIn0d001_norm_0n1.csv"
diDEFSDf.to_csv(savePath)